# **Install the pyAFL model**

Run this if the pyAFL library has not been installed

In [ ]:
pip install pyAFL

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 KB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 KB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 KB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.8/316.8 KB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 KB 892.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.8/227.8 KB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

# Libraries

In [ ]:
import pandas as pd

from pyAFL.seasons.models import Season

from bs4 import BeautifulSoup
from datetime import datetime
import requests

pd.set_option('display.max_columns', None)

# **Functions for Data Load**

In [ ]:
def add_leading_zero(code):
    if code < 10:
        code = "0" + str(code)
    return code

In [ ]:
def get_season_stats(start_year, end_year):
    '''
    Function to collect multiple seasons of data in one request
    '''

    games = pd.DataFrame()
    
    for year in range(start_year, end_year+1):
        season = Season(year)
        stats = season.get_season_stats()
        stats_match = stats.match_summary
        games = pd.concat([games, stats_match], axis=0)

    games.reset_index(inplace=True, drop=True)

    return games

In [ ]:
def get_table_hist(start_year, end_year, end=23):
    '''

    Parameters
    ----------
    start_year : Date
        The starting year that you wish to extract data for.
    end_year : Date
        The end year that you wish to extract data for.
    end : Integer, optional
        The maximum round that you wish to extract to if you wish it to be shorter than a full season. The default is 23.

    Returns
    -------
    A dataframe history of the AFL ladder at the end of each round.

    '''

    rd = 1
    end = end

    df1 = pd.DataFrame()

    for year in range(start_year, end_year+1):

        while rd <= end:

            url = requests.get(
                f'https://finalsiren.com/AFLLadder.asp?AFLLadderTypeID=2&SeasonID={year}&Round={rd}-1')
            dfs = pd.read_html(url.text)
            data = pd.DataFrame(data=dfs[0])

            if rd == 1:
                data.columns = ['Pos', 'Team', 'P', 'W', 'D', 'L', 'For', 'Agn', 'Max', 'Min', 'Home_W', 'Home_D', 'Home_L', 'Away_W',
                                'Away_D', 'Away_L', 'Stk', 'Pts', '%', 'Pos12', 'W12', 'D12', 'L12', 'Pts12', '12%']
                #data['Chg'] = 'N'
            else:
                data.columns = ['Pos', 'Team', 'P', 'W', 'D', 'L', 'For', 'Agn', 'Max', 'Min', 'Home_W', 'Home_D', 'Home_L', 'Away_W',
                                'Away_D', 'Away_L', 'Stk', 'Chg', 'Pts', '%', 'Pos12', 'W12', 'D12', 'L12', 'Pts12', '12%']
                data = data.drop('Chg', axis=1)

            rstring = str(rd+1)
            data['Round'] = rstring
            data['Year'] = year

            df1 = pd.concat((df1,data), axis=0)
            print(f'Round {rd} Season {year} is completed')

            rd += 1
        
        rd = 1

    return df1 

In [ ]:
def clean_table_data(df):
    '''
    Function to clean the table dataframe
    '''

    # Pre process table data
    df['Round'] = df['Round'].astype(int)
    df['Team'] = df['Team'].replace('GWS Giants','Greater Western Sydney')
    
    # Drop 12 month ago columns
    df = df.drop(['Pos12', 'W12', 'D12', 'L12', 'Pts12', '12%'], axis=1)

    # Strip out the for and against value
    df['For'] = df['For'].str.replace(r" \(.*\)", "")
    df['Agn'] = df['Agn'].str.replace(r" \(.*\)", "")

    # Fix the Streak column
    df[['Stkn', 'Stkd']] = df['Stk'].str.split('(\d+)([A-Za-z]+)', expand=True).loc[:, [1, 2]]
    
    # drop NA rows
    df = df.dropna()

    # Convert the Stkn to an int and then mulitple a loss by negative to create a minus value
    df['Stkn'] = df['Stkn'].astype(int)
    df.loc[df['Stkd'] == 'L', 'Stkn'] *= -1
    df.loc[df['Stkd'] == 'D', 'Stkn'] = 0
    return df

In [ ]:
def clean_games_data(games_df, team_code_df, table_hist_df):
    '''
    Function to clean the games data and join with the table dataframe
    '''
    
    # Fix team names
    team_name_fixes = {'Footscray': 'Western Bulldogs', 'Kangaroos': 'North Melbourne', 'South Melbourne':'Sydney'}
    games_df = games_df.replace(team_name_fixes)

    # Fix the round for finals games to join to the table
    games_df['Round'] = games_df['Round'].apply(lambda x: 24 if x > 24 else x)

    # Tag the early part of the season to account for variability from season to season
    games_df['Year stage'] = games_df.apply(lambda x: 'Finals' if 'Final' in x['Year stage'] else('Early season' if x['Round'] < 4 else x['Year stage']), axis=1)

    # Split the Date time for match stats fetch
    games_df[['day', 'month', 'year']] = games_df['Date'].dt.strftime('%d-%m-%Y').str.split('-', expand=True)
    games_df['time'] = games_df['Date'].dt.time
    games_df['year'] = games_df['year'].astype(int)

    # Add a leading zero to month & day
    games_df['month'] = games_df['month'].apply(lambda x: x.zfill(2))
    games_df['day'] = games_df['day'].apply(lambda x: x.zfill(2))

    # Add two columns to enable a join of the first round to the last round
    games_df['round_key'] = games_df['Round'].apply(lambda x: 24 if x == 1 else x)
    games_df['year_key'] = games_df.apply(lambda x: x['year'] - 1 if x['Round'] == 1 else x['year'], axis=1)
    
    # Merge all the tables together in preparation for the detailed stats
    games_df = pd.merge(games_df, team_code_df[['Team','Code']], left_on='Home team', right_on='Team', how='left')
    games_df = games_df.drop('Team', axis=1)
    games_df = games_df.rename(columns={'Code':'HomeCode'})

    games_df = pd.merge(games_df, team_code_df[['Team','Code']], left_on='Away Team', right_on='Team', how='left')
    games_df = games_df.drop('Team', axis=1)
    games_df = games_df.rename(columns={'Code':'AwayCode'})

    games_df = pd.merge(games_df, table_hist_df, left_on=['Home team','round_key','year_key'], right_on=['Team','Round','Year'], how='left')
    games_df = pd.merge(games_df, table_hist_df, left_on=['Away Team','round_key','year_key'], right_on=['Team','Round','Year'], suffixes=('_home','_away'), how='left')

    games_df = games_df.dropna(subset=['Team_home'])
    games_df.reset_index(inplace=True)
    
    return games_df

# **Data Load**

In [ ]:
games = get_season_stats(start_year=2022, end_year=2022)

table_hist = get_table_hist(start_year=2022, end_year=2022)

Round 1 Season 2022 is completed
Round 2 Season 2022 is completed
Round 3 Season 2022 is completed
Round 4 Season 2022 is completed
Round 5 Season 2022 is completed
Round 6 Season 2022 is completed
Round 7 Season 2022 is completed
Round 8 Season 2022 is completed
Round 9 Season 2022 is completed
Round 10 Season 2022 is completed
Round 11 Season 2022 is completed
Round 12 Season 2022 is completed
Round 13 Season 2022 is completed
Round 14 Season 2022 is completed
Round 15 Season 2022 is completed
Round 16 Season 2022 is completed
Round 17 Season 2022 is completed
Round 18 Season 2022 is completed
Round 19 Season 2022 is completed
Round 20 Season 2022 is completed
Round 21 Season 2022 is completed
Round 22 Season 2022 is completed
Round 23 Season 2022 is completed


In [ ]:
# Team Code data for joins between datasets
data = {'Team': ['Adelaide', 'Brisbane Lions', 'Brisbane Bears', 'Carlton', 'Collingwood', 'Essendon', 'Fitzroy', 'Fremantle', 'Geelong', 'Gold Coast', 'Greater Western Sydney', 'Hawthorn', 'Melbourne', 'North Melbourne', 'Port Adelaide', 'Richmond', 'St Kilda', 'Sydney', 'University', 'West Coast', 'Western Bulldogs'],
        'Code': [1, 19, 2, 3, 4, 5, 6, 8, 9, 20, 21, 10, 11, 12, 13, 14, 15, 16, 17, 18, 7],
        'Abv': ['AD', 'BL', 'BB', 'CA', 'CW', 'ES', 'FI', 'FR', 'GE', 'GC', 'GW', 'HW', 'ME', 'NM', 'PA', 'RI', 'SK', 'SY', 'UN', 'WC', 'WB']}

team_code = pd.DataFrame(data)
team_code['Code'] = team_code['Code'].apply(add_leading_zero)

In [ ]:
# Create a copy of the table to complete data cleaning & join
table_hist_2 = table_hist.copy()

In [ ]:
# Run the function to clean the table data ready to be joined
table_hist_2 = clean_table_data(table_hist_2)

<ipython-input-7-4b7b90ac1cfc>:14: FutureWarning: The default value of regex will change from True to False in a future version.
  df['For'] = df['For'].str.replace(r" \(.*\)", "")
<ipython-input-7-4b7b90ac1cfc>:15: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Agn'] = df['Agn'].str.replace(r" \(.*\)", "")


In [ ]:
# Run the function to clean the games data and then join the table and games data
cleaned_data = clean_games_data(games, team_code, table_hist_2)

In [ ]:
cleaned_data.head()

,index,Date,Round_x,Game number,Venue,Home team,Away Team,Home team score,Away team score,Home team score detail,Away team score detail,Winning team,Margin,Year stage,day,month,year,time,round_key,year_key,HomeCode,AwayCode,Pos_home,Team_home,P_home,W_home,D_home,L_home,For_home,Agn_home,Max_home,Min_home,Home_W_home,Home_D_home,Home_L_home,Away_W_home,Away_D_home,Away_L_home,Stk_home,Pts_home,%_home,Round_y,Year_home,Stkn_home,Stkd_home,Pos_away,Team_away,P_away,W_away,D_away,L_away,For_away,Agn_away,Max_away,Min_away,Home_W_away,Home_D_away,Home_L_away,Away_W_away,Away_D_away,Away_L_away,Stk_away,Pts_away,%_away,Round,Year_away,Stkn_away,Stkd_away
0,9,2022-03-24 18:20:00,2,1,Docklands,Western Bulldogs,Carlton,90,102,"[4, 1, 7, 3, 11, 5, 13, 12]","[5, 2, 12, 4, 14, 5, 16, 6]",Carlton,12,Early season,24,03,2022,18:20:00,2,2022,07,03,17.0,Western Bulldogs,1.0,0.0,0.0,1.0,71,97,71.0,71.0,0.0,0.0,0.0,0.0,0.0,1.0,1L,0.0,73.20,2.0,2022.0,-1.0,L,5.0,Carlton,1.0,1.0,0.0,0.0,101,76,101.0,101.0,1.0,0.0,0.0,0.0,0.0,0.0,1W,4.0,132.89,2.0,2022.0,1.0,W
1,10,2022-03-25 18:50:00,2,2,S.C.G.,Sydney,Geelong,107,77,"[4, 3, 11, 3, 15, 4, 17, 5]","[2, 4, 6, 7, 8, 13, 10, 17]",Sydney,30,Early season,25,03,2022,18:50:00,2,2022,16,09,6.0,Sydney,1.0,1.0,0.0,0.0,112,92,112.0,112.0,0.0,0.0,0.0,1.0,0.0,0.0,1W,4.0,121.74,2.0,2022.0,1.0,W,1.0,Geelong,1.0,1.0,0.0,0.0,138,72,138.0,138.0,1.0,0.0,0.0,0.0,0.0,0.0,1W,4.0,191.67,2.0,2022.0,1.0,W
2,11,2022-03-26 12:45:00,2,3,M.C.G.,Collingwood,Adelaide,100,58,"[5, 3, 7, 5, 14, 7, 15, 10]","[1, 5, 3, 6, 7, 6, 8, 10]",Collingwood,42,Early season,26,03,2022,12:45:00,2,2022,04,01,7.0,Collingwood,1.0,1.0,0.0,0.0,102,85,102.0,102.0,0.0,0.0,0.0,1.0,0.0,0.0,1W,4.0,120.00,2.0,2022.0,1.0,W,10.0,Adelaide,1.0,0.0,0.0,1.0,82,83,82.0,82.0,0.0,0.0,1.0,0.0,0.0,0.0,1L,0.0,98.80,2.0,2022.0,-1.0,L
3,12,2022-03-26 15:35:00,2,4,Docklands,Essendon,Brisbane Lions,75,97,"[4, 5, 5, 9, 8, 13, 10, 15]","[1, 1, 7, 2, 13, 5, 15, 7]",Brisbane Lions,22,Early season,26,03,2022,15:35:00,2,2022,05,19,18.0,Essendon,1.0,0.0,0.0,1.0,72,138,72.0,72.0,0.0,0.0,0.0,0.0,0.0,1.0,1L,0.0,52.17,2.0,2022.0,-1.0,L,8.0,Brisbane Lions,1.0,1.0,0.0,0.0,80,69,80.0,80.0,1.0,0.0,0.0,0.0,0.0,0.0,1W,4.0,115.94,2.0,2022.0,1.0,W
4,13,2022-03-26 18:40:00,2,5,Adelaide Oval,Port Adelaide,Hawthorn,56,120,"[0, 3, 3, 6, 7, 10, 7, 14]","[3, 2, 8, 4, 14, 4, 19, 6]",Hawthorn,64,Early season,26,03,2022,18:40:00,2,2022,13,10,11.0,Port Adelaide,1.0,0.0,0.0,1.0,69,80,69.0,69.0,0.0,0.0,0.0,0.0,0.0,1.0,1L,0.0,86.25,2.0,2022.0,-1.0,L,3.0,Hawthorn,1.0,1.0,0.0,0.0,78,58,78.0,78.0,1.0,0.0,0.0,0.0,0.0,0.0,1W,4.0,134.48,2.0,2022.0,1.0,W
